<a href="https://colab.research.google.com/github/robonesky/oreilly/blob/main/RM%20Lab%20Notes%2012%20202301%20cohort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=5b29f54d9de8b395cc6d20463c33d7407272158c3ad7b166721cae7dc3ea5846
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 1000)

np.random.seed(42)

dow = pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average')

dow

[                                                   0                                                  1
 0  Historical logarithmic graph of the DJIA from ...  Historical logarithmic graph of the DJIA from ...
 1                                         Foundation  February 16, 1885; 138 years ago[1] May 26, 18...
 2                                           Operator                              S&P Dow Jones Indices
 3                                          Exchanges  .mw-parser-output .plainlist ol,.mw-parser-out...
 4                                     Trading symbol                                  ^DJI$INDU.DJIDJIA
 5                                       Constituents                                                 30
 6                                               Type                                          Large cap
 7                                         Market cap           US$10.9 trillion (as of May 31, 2023)[3]
 8                                   Weighting method  

In [4]:
# the components table is the 2nd table:

dow[1]

,Company,Exchange,Symbol,Industry,Date added,Notes,Index weighting
0,3M,NYSE,MMM,Conglomerate,1976-08-09,As Minnesota Mining and Manufacturing,2.41%
1,American Express,NYSE,AXP,Financial services,1982-08-30,NaN,3.02%
2,Amgen,NASDAQ,AMGN,Biopharmaceutical,2020-08-31,NaN,5.48%
3,Apple,NASDAQ,AAPL,Information technology,2015-03-19,NaN,2.84%
4,Boeing,NYSE,BA,Aerospace and defense,1987-03-12,NaN,3.36%
5,Caterpillar,NYSE,CAT,Construction and Mining,1991-05-06,NaN,4.52%
6,Chevron,NYSE,CVX,Petroleum industry,2008-02-19,Also 1930-07-18 to 1999-11-01,3.50%
7,Cisco,NASDAQ,CSCO,Information technology,2009-06-08,NaN,0.96%
8,Coca-Cola,NYSE,KO,Drink industry,1987-03-12,Also 1932-05-26 to 1935-11-20,1.22%
9,Disney,NYSE,DIS,Broadcasting and entertainment,1991-05-06,NaN,1.89%


In [5]:
# Now get the list of stock symbols:
dow_stocks = list(dow[1]['Symbol'])
dow_company_name = list(dow[1]['Company'])
dow_stocks[:5]

['MMM', 'AXP', 'AMGN', 'AAPL', 'BA']

In [6]:
dow_company_name[:5]

['3M', 'American Express', 'Amgen', 'Apple', 'Boeing']

In [7]:
# If you wanted to download direct from yahoo finance:
######################################################

# Use comprehension to fetch data for multiple fields
dow_ohlc = {symbol: yf.download(symbol, start='2010-01-01', end='2022-05-31', auto_adjust=True, progress=False) for symbol in dow_stocks}

# Save dow_ohlc dict for future use
np.save('./dow_ohlc.npy', dow_ohlc)



In [9]:
# get the open close high low data:

ohlc = np.load('./dow_ohlc.npy', allow_pickle = True).item()
ohlc['MMM'].head()

,Open,High,Low,Close,Volume
Date,,,,,
2010-01-04,55.903347,56.145557,55.620770,55.856251,3043700
2010-01-05,55.708252,55.997559,54.968163,55.506409,2847000
2010-01-06,56.434852,56.919272,56.185918,56.293564,5268500
2010-01-07,56.058098,56.354133,55.250734,56.333950,4470100
2010-01-08,56.307057,56.730923,56.044664,56.730923,3405800


In [10]:
for symbol in dow_stocks:
  high = ohlc[symbol]['High']
  low = ohlc[symbol]['Low']
  close = ohlc[symbol]['Close']

  # not entirely clear what 'atr' is - "Average true Range" it seems.  It's a volatility thing
  ohlc[symbol]['ATR'] = ta.atr(high, low, close)

In [14]:
ohlc['MMM'].head()

# We'll have to sort out these NaNs later

,Open,High,Low,Close,Volume,ATR
Date,,,,,,
2010-01-04,55.903347,56.145557,55.620770,55.856251,3043700,NaN
2010-01-05,55.708252,55.997559,54.968163,55.506409,2847000,NaN
2010-01-06,56.434852,56.919272,56.185918,56.293564,5268500,NaN
2010-01-07,56.058098,56.354133,55.250734,56.333950,4470100,NaN
2010-01-08,56.307057,56.730923,56.044664,56.730923,3405800,NaN


In [16]:
# returns

stock_price = pd.DataFrame(
    {symbol: ohlc[symbol]['Close']
    for symbol
    in dow_stocks}
)

stock_ret = stock_price.resample('W-FRI').last().pct_change()

stock_ret.head()



,MMM,AXP,AMGN,AAPL,BA,CAT,CVX,CSCO,KO,DIS,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,MCD,MRK,MSFT,NKE,PG,CRM,TRV,UNH,VZ,V,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-15,-0.011267,0.010489,-0.009159,-0.028540,-0.012663,0.003151,-0.003020,-0.010544,0.020671,-0.040150,NaN,-0.052206,-0.014148,0.012108,0.007107,-0.001440,0.005451,-0.022382,0.007115,0.046950,0.006523,-0.020238,0.006287,-0.074444,0.004737,0.032110,-0.036850,-0.009885,-0.010543,0.006563
2010-01-22,-0.022670,-0.089644,0.006222,-0.039722,-0.050148,-0.097638,-0.058563,-0.058606,-0.036418,-0.020261,NaN,-0.067126,-0.029751,-0.064509,-0.047655,-0.042788,-0.021066,-0.103480,0.017823,-0.015201,-0.061568,-0.021897,-0.001266,-0.059595,-0.009838,-0.017482,-0.007849,-0.039587,-0.016667,-0.013785
2010-01-29,-0.012151,-0.024099,0.033216,-0.028774,0.048987,-0.037050,-0.033114,-0.021768,0.000184,-0.014343,NaN,-0.035038,0.010461,-0.031093,-0.024781,-0.025615,-0.005380,-0.005617,-0.015144,-0.017752,-0.026934,0.012226,0.020561,-0.015339,0.048851,-0.004824,-0.030323,-0.008461,0.001667,0.009256
2010-02-05,-0.024226,0.005045,-0.013680,0.017703,-0.029724,-0.009380,-0.013034,0.054739,-0.021383,-0.000338,NaN,0.036579,-0.001071,-0.030279,0.009233,0.011636,-0.003500,-0.016436,0.015057,-0.037978,-0.005678,-0.032157,-0.004062,-0.023131,-0.007697,-0.013940,-0.023114,0.006461,-0.076006,0.000374


In [17]:
stock_ret.isnull().sum()

MMM       1
AXP       1
AMGN      1
AAPL      1
BA        1
CAT       1
CVX       1
CSCO      1
KO        1
DIS       1
DOW     481
GS        1
HD        1
HON       1
IBM       1
INTC      1
JNJ       1
JPM       1
MCD       1
MRK       1
MSFT      1
NKE       1
PG        1
CRM       1
TRV       1
UNH       1
VZ        1
V         1
WBA       1
WMT       1
dtype: int64